## Convert JSON Data to Markdown Using Python

> This guide explains how to use Python to convert JSON data into Markdown format. It is useful for transforming structured data into readable documentation.

## Fetch JSON Data

### Obtain an API Key

Apply for an API key from the official website: [EASYDOC](https://easydoc.sh/zh)
- Log in and click on "API Keys" in the left menu.
- Click "Create API Key" to generate an API key.

In [ ]:
APIKEY = "your API key here"  # your API key here

### Send a Parse Request

In [ ]:
import requests  # pip install requests

url = "https://api.easydoc.sh/api/v1/parse"
headers = {
    "api-key": APIKEY
}
files = {
    "file": open('your file path', "rb")
}
data = {
    "mode": "premium"   # Required for node structure relationship data
}

response = requests.post(url, headers=headers, files=files, data=data)
print(response.json())

{'data': {'task_id': 'parse_76a8e227-2e1c-4e51-83df-bf5c6d552f85'}, 'success': True}


### Retrieve the Parsing Result

In [37]:
import requests

task_id = response.json().get("data", {}).get("task_id")
url = f"https://api.easydoc.sh/api/v1/parse/{task_id}/result"
headers = {
    "api-key": APIKEY
}

response = requests.get(url, headers=headers)
json_data = response.json()
print(json_data)  # JSON data response

{'data': {'task_id': 'parse_76a8e227-2e1c-4e51-83df-bf5c6d552f85', 'task_result': {'file_name': '财务报告-报表_5.pdf', 'nodes': [{'id': 1, 'text': '证券代码：300757 证券简称：罗博特科 公告编号：2024-036', 'type': 'Text', 'relation': 'part_of', 'parent_id': -1, 'composing_blocks': [{'system': 'PixelSpace', 'coordinates': [315.8567810058594, 526.1871948242188, 2620.138427734375, 607.2030029296875], 'page_number': 1, 'layout_width': 2985, 'layout_height': 4220}]}, {'id': 2, 'text': '罗博特科智能科技股份有限公司\n2024年第一季度报告', 'type': 'Title', 'relation': 'part_of', 'parent_id': -1, 'composing_blocks': [{'system': 'PixelSpace', 'coordinates': [941.9064331054688, 858.51611328125, 1984.0863037109375, 1100.014404296875], 'page_number': 1, 'layout_width': 2985, 'layout_height': 4220}]}, {'id': 3, 'text': '本公司及董事会全体成员保证信息披露的内容真实、准确、完整，没有虚假记载、误\n导性陈述或重大遗漏', 'type': 'Text', 'relation': 'part_of', 'parent_id': 2, 'composing_blocks': [{'system': 'PixelSpace', 'coordinates': [322.00018310546875, 1168.36865234375, 2596.83642578125, 1344.1

## Convert Parsed JSON Data to Markdown Format

### Define Parsing Functions

In [38]:
def rule(node, depth):
    txt = ""
    if node["type"] == "Text":
        txt = node["text"]
    elif node["type"] == "Title":
        title_level = "#" * depth
        txt = f"{title_level} {node['text']}"
    elif node["type"] in ("Table", "Figure"):
        txt = f"""\n```{"table" if node["type"] == "Table" else "figure"}\n"""
        txt += node.get("vlm_understanding", node["text"]) + "\n```\n"
        # if node.get("caption"):
        #     txt = f"""<div style="text-align: right;">{node.get('caption')}</div>\n""" + txt
        # if node.get("note"):
        #     txt = f"""<div style="text-align: left;">{node.get('note')}</div>\n""" + txt
    return f"\n{txt}\n"


def tree_flat(tree, depth=1):
    rst = ""
    for node in tree:
        rst += rule(node, depth)
        if node.get("type") == "Title":
            rst += tree_flat(node["children"], depth + 1)
    return rst


def build_tree(nodes, parent_id=-1):
    tree = []
    for node in nodes:
        if node["parent_id"] == parent_id:
            children = build_tree(nodes, node["id"])
            node["children"] = children
            tree.append(node)
    return tree

### Convert Data into Tree Structure

In [39]:
tree = build_tree(json_data.get("data", {}).get("task_result", {}).get("nodes", []))
print(tree)

[{'id': 1, 'text': '证券代码：300757 证券简称：罗博特科 公告编号：2024-036', 'type': 'Text', 'relation': 'part_of', 'parent_id': -1, 'composing_blocks': [{'system': 'PixelSpace', 'coordinates': [315.8567810058594, 526.1871948242188, 2620.138427734375, 607.2030029296875], 'page_number': 1, 'layout_width': 2985, 'layout_height': 4220}], 'children': []}, {'id': 2, 'text': '罗博特科智能科技股份有限公司\n2024年第一季度报告', 'type': 'Title', 'relation': 'part_of', 'parent_id': -1, 'composing_blocks': [{'system': 'PixelSpace', 'coordinates': [941.9064331054688, 858.51611328125, 1984.0863037109375, 1100.014404296875], 'page_number': 1, 'layout_width': 2985, 'layout_height': 4220}], 'children': [{'id': 3, 'text': '本公司及董事会全体成员保证信息披露的内容真实、准确、完整，没有虚假记载、误\n导性陈述或重大遗漏', 'type': 'Text', 'relation': 'part_of', 'parent_id': 2, 'composing_blocks': [{'system': 'PixelSpace', 'coordinates': [322.00018310546875, 1168.36865234375, 2596.83642578125, 1344.1488037109375], 'page_number': 1, 'layout_width': 2985, 'layout_height': 4220}], 'children': []

### Convert the Tree Structure into Markdown Format

In [40]:
rst = tree_flat(tree)
print(rst)


证券代码：300757 证券简称：罗博特科 公告编号：2024-036

# 罗博特科智能科技股份有限公司
2024年第一季度报告

本公司及董事会全体成员保证信息披露的内容真实、准确、完整，没有虚假记载、误
导性陈述或重大遗漏

## 重要内容提示：

1.董事会、监事会及董事、监事、高级管理人员保证季度报告的真实、准确、完整，不存
在虚假记载、误导性陈述或重大遗漏，并承担个别和连带的法律责任。

2.公司负责人、主管会计工作负责人及会计机构负责人（会计主管人员）声明：保证季度报
告中财务信息的真实、准确、完整。

3.第一季度报告是否经过审计

口是否

## 一、主要财务数据

### （一）主要会计数据和财务指标

公司是否需追溯调整或重述以前年度会计数据

口是否


```figure
这张图片显示的是一份财务报告，其中包含了营业收入、归属于上市公司股东的净利润、归属于上市公司股东的扣除非经常性损益后的净利润等各项财务指标的数据对比。具体数据如下：

- 本报告期营业收入为2,029,838,490元，比上年同期增长1.52%。
- 本报告期归属于上市公司股东的净利润为5,898,882,467元，比上年同期增长39.23%。
- 本报告期归属于上市公司股东的扣除非经常性损益后的净利润为5,708,214,767元，比上年同期增长30.12%。
- 本报告期经营活动产生的现金流量净额为-26,763,463,58元，比上年同期减少346.98%。

此外，报告还列出了基本每股收益、稀释每股收益和加权平均净资产收益率等财务指标的数据对比。这些数据表明公司在过去的一段时间内取得了显著的增长，但也需要注意经营活动现金流量净额的大幅下降。
```


### （二）非经常性损益项目和金额

适用口不适用

单位：元


```table
项目
 本报告期金额
 说明

非流动性资产处置损益（包括已计提

26,418.39

资产减值准备的冲销部分）

计入当期损益的政府补助（与公司正
 常经营业务密切相关、符合国家政策

176,228.26

规定、按照确定的标准享有、对公司
 损益产生持续影响的政府补助除外）
 除上述各项之外的其他营业外收入和

33,055.18

支出

减：所得税影响额
 45,597.02

少数股东权益影响额（税后）
 257